In [168]:
import pandas as pd
import numpy as np
import glob, sys, os, time , itertools , warnings
from bs4 import BeautifulSoup
from tqdm import tqdm
import multiprocessing as mp
# from DataProcessing import 

warnings.filterwarnings('ignore')

In [169]:
state_codes = pd.read_excel('/home/vlad/csv/Us-zipcodes/statecode.xlsx',index_col=0)

In [170]:
#Unemployment

path_to_unemp = '/home/vlad/csv/US-Unemployement/us-county-unemployement20142018.xlsx'

unempdf = pd.concat([pd.read_excel(path_to_unemp,sheet_name=i,index_col=0) for i in range(4)],axis=1)

col_to_filt = [col for col in unempdf.columns if 'Pct' in col]

unempdf= unempdf[col_to_filt]
unempdf.columns = ['Unemp-'+col for col in unempdf.columns]

unempdf.index = [', '.join(ind.replace('County','').replace(' ','').strip().split(',')) 
                 for ind in unempdf.index]
unempdf.sort_index()

,Unemp-Pct-2014,Unemp-Pct-2015,Unemp-Pct-2016,Unemp-Pct-2017
"Abbeville, SC",7.7,6.9,5.5,4.6
"AcadiaParish, LA",5.8,6.8,7.4,5.9
"Accomack, VA",6.7,5.4,4.9,4.5
"Ada, ID",4.1,3.7,3.3,2.8
"Adair, IA",4.0,3.2,2.9,2.8
...,...,...,...,...
"Yuma, AZ",22.9,21.7,18.9,17.0
"Yuma, CO",3.3,2.6,2.1,1.5
"Zapata, TX",5.2,7.6,10.8,7.4
"Zavala, TX",12.1,11.2,13.9,11.2


In [171]:
#Income

path_to_income = '/home/vlad/csv/US-Personal-Income/lapi1118_0.xlsx'
income = (pd.read_excel(path_to_income
                       #,sheet_name=1
                       ,skiprows=3
#                        ,index_col=0
                       ,names = ['County',	'income-2015',	'income-2016'
                                ,'income-2017'
                                ,2017.1,2016.1,	2017.2,	2017.3]
                      )
         .iloc[:3216#,:4
              ]
             
         )

df_list = [df.iloc[1:] for df in np.split(income,income[income.isnull().all(1)].index)][1:]

df_list_res = []
for i in range(len(df_list)):
    #print(i)
    df_list_res.append(df_list[i].assign(State = lambda x: x.iloc[0,0]).iloc[1:]
                      )
income_df = pd.concat(df_list_res,ignore_index=True)
income_df['State-Code'] =income_df['State'].map(lambda x: state_codes.loc[x,'Postal Code'])
# income_df['County'] = income_df['County'].map(str) +' County'

income_df['County-State'] = income_df['County'].map(str) +', '+ income_df['State-Code'].map(str)
income_df = income_df.set_index('County-State').iloc[:,:-2]
income_df.sort_index()

,County,income-2015,income-2016,income-2017,2017.1,2016.1,2017.2,2017.3
County-State,,,,,,,,
"Abbeville, SC",Abbeville,30928.0,31785.0,33044.0,30,2.8,4.0,14
"Acadia, LA",Acadia,35676.0,34158.0,35401.0,48,-4.3,3.6,24
"Accomack, VA",Accomack,38807.0,39099.0,40701.0,57,0.8,4.1,28
"Ada, ID",Ada,47671.0,48419.0,49900.0,3,1.6,3.1,23
"Adair, IA",Adair,47128.0,47917.0,46459.0,44,1.7,-3.0,91
...,...,...,...,...,...,...,...,...
"Yuma, AZ",Yuma,31847.0,33259.0,34752.0,9,4.4,4.5,5
"Yuma, CO",Yuma,54543.0,52271.0,55197.0,17,-4.2,5.6,12
"Zapata, TX",Zapata,29459.0,25690.0,27325.0,248,-12.8,6.4,40


In [111]:
path_to_pop = "/home/vlad/csv/US-Population/cbsa-est2018-alldata.xlsx"
pop = pd.read_excel(path_to_pop,sheet_name=1#,index_col=0
                     )
pop['NAME'] =pop['NAME'].str.strip()
pop = pop.set_index('NAME')
pop.index = [', '.join(ind.replace('County','').replace(' ','').strip().split(',')) for ind in pop.index]
pop.sort_index()

,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018
"Abbeville, SC",25417,25399,25310,25060,25000,24880,24775,24775,24633,24560,24541
"Aberdeen, SD",40602,40603,40717,40962,41576,42155,42220,42404,42849,43235,43191
"Aberdeen, WA",72797,72798,72843,72357,71770,71046,70783,71027,71572,72553,73901
"Abilene, TX",165252,165246,165583,166616,167447,167472,168355,169704,170018,170516,171451
"AcadiaParish, LA",61773,61787,61875,61853,61991,62294,62664,62681,62793,62514,62190
...,...,...,...,...,...,...,...,...,...,...,...
"Yuma, AZ",195751,195750,197138,202883,202589,202589,204283,205456,207520,209756,212128
"Yuma, AZ",195751,195750,197138,202883,202589,202589,204283,205456,207520,209756,212128
"Zanesville, OH",86074,86086,86213,86236,85786,85640,85900,86174,85973,86148,86183
"Zapata, TX",14018,14018,14087,14229,14285,14403,14404,14511,14456,14283,14190


In [138]:
joint_full = unempdf.join(income_df).join(pop)

In [164]:
joint_full_nn = joint_full.loc[~joint_full.isnull().any(1)].drop(columns=['County'])

In [165]:
joint_full.loc['Cook, IL']

Unemp-Pct-2014               7.5
Unemp-Pct-2015               6.2
Unemp-Pct-2016                 6
Unemp-Pct-2017               5.1
County                      Cook
income-2015                56470
income-2016                57285
income-2017                59238
CENSUS2010POP        5.19468e+06
ESTIMATESBASE2010    5.19503e+06
POPESTIMATE2010      5.19917e+06
POPESTIMATE2011      5.21998e+06
POPESTIMATE2012      5.24050e+06
POPESTIMATE2013      5.25488e+06
POPESTIMATE2014      5.25748e+06
POPESTIMATE2015      5.24766e+06
POPESTIMATE2016      5.22846e+06
POPESTIMATE2017       5.2045e+06
POPESTIMATE2018      5.18049e+06
Name: Cook, IL, dtype: object

In [167]:
path_to_output ='/home/vlad/csv/SC-output/'

joint_full_nn.reset_index().to_csv(path_to_output+'demography.csv',index=False)